# Notebook for reviews scrapping from MyAnimeList

In [1]:
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import random
import json
import pathlib
import os

In [2]:
def get_reviews_html(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'}
    response = requests.get(url, headers=headers)
    html_page = BeautifulSoup(response.text, 'html.parser')
    reviews_html = html_page.find_all(class_='review-element js-review-element')
    return reviews_html

In [3]:
class Review:
  def __init__(self, html):
    self.html = html

  def update_date(self):
    self.update_date = (self.html
                        .find(class_='update_at')
                        .get_text(strip=True))
    pass

  def user_name(self):
    self.user_name = (self.html
                      .find(class_='username')
                      .get_text(strip=True))
    pass
  
  def anime_id(self):
    self.anime_id = (self.html
                     .find(class_='titleblock mb4')
                     .find('a')
                     .get('href')
                     .split('https://myanimelist.net/anime/')[1]
                     .split('/')[0])
    pass
 
  def anime(self):
    self.anime = (self.html
                  .find(class_='titleblock mb4')
                  .find('a')
                  .get_text(strip=True))
    pass
  
  def tags(self):
    self.tags = (self.html
                 .find(class_='tags')
                 .get_text(';', strip=True)
                 .split(';'))
    pass
  
  def text(self):
    self.text = (self.html
                 .find(class_='text')
                 .get_text(';', strip=True))
    pass

  def get_data(self):
    self.update_date()
    self.user_name()
    self.anime_id()
    self.anime()
    self.tags()
    self.text()
    pass


In [25]:
def scrape_reviews(pages):
    reviews = {}
    for page in tqdm(pages):
        url = f'https://myanimelist.net/reviews.php?t=anime&filter_check=&filter_hide=&preliminary=on&spoiler=off&p={page}'
        reviews_html = get_reviews_html(url)
        page_reviews = []

        for html in reviews_html:
            review = Review(html)
            review.get_data()
            data = vars(review)
            data.pop('html')
            page_reviews.append(data)
        
        reviews[page] = page_reviews
            
        time.sleep(random.randint(2,6))
    
    return reviews       

In [27]:
dir = os.path.join(os.getcwd(), 'data\\reviews')
pathlib.Path(dir).mkdir(parents=True, exist_ok=True)

reviews = scrape_reviews(pages=[1,2,3])

with open(f'{dir}\\reviews.json', 'w') as f:
    json.dump(reviews, f)

100%|██████████| 3/3 [00:16<00:00,  5.52s/it]
